In [1]:
from liberty.parser import parse_liberty
from liberty.types import *
import numpy as np
import sys

from std_lib_tk.util import *
from std_lib_tk.combinatorial import *
from std_lib_tk.sequential import *

In [2]:
lib = parse_liberty(open("./../data/D_CELLS_HD_LP5MOS_slow_1_08V_175C.lib").read())

### Testbenches for Combinatorial cells

In [3]:
posedge_flip_flops      = {str(cell.args[0]).strip('"') for cell in lib.get_groups('cell') if 'Flip-Flop' in attribute2dict(cell.attributes)['cell_description'] and 'posedge' in attribute2dict(cell.attributes)['cell_description'] and ('Scan' not in attribute2dict(cell.attributes)['cell_description'])}

negedge_flip_flops      = {str(cell.args[0]).strip('"') for cell in lib.get_groups('cell') if 'Flip-Flop' in attribute2dict(cell.attributes)['cell_description'] and 'negedge' in attribute2dict(cell.attributes)['cell_description']}

adder                   = {str(cell.args[0]).strip('"') for cell in lib.get_groups('cell') if 'Adder' in attribute2dict(cell.attributes)['cell_description']}

muxer                   = {str(cell.args[0]).strip('"') for cell in lib.get_groups('cell') if 'Multiplexer' in attribute2dict(cell.attributes)['cell_description']}

logic                   = {str(cell.args[0]).strip('"') for cell in lib.get_groups('cell')
                            if 'AND' in attribute2dict(cell.attributes)['cell_description'] 
                            or 'OR' in attribute2dict(cell.attributes)['cell_description'] 
                            or (('Buffer' in attribute2dict(cell.attributes)['cell_description']) and ('latch' not in attribute2dict(cell.attributes)['cell_description']) and ('Clock' not in attribute2dict(cell.attributes)['cell_description']) and ('Tristate' not in attribute2dict(cell.attributes)['cell_description']))
                            or ('Inverter' in attribute2dict(cell.attributes)['cell_description'] and ('Tristate' not in attribute2dict(cell.attributes)['cell_description']))
                            } 

In [4]:
cells = [select_cell(lib, name) for name in logic if not name.startswith("BUSSHD")] # remove dont use cell

In [5]:
path = "./netlists/tb_"

# Testbenches for all cells in logic

# for cell in cells:

#     write_cell_testbench(cell=cell, v_sup=0.8, temperature=175, tran_time=12e-9, path=path) 

cell = select_cell(lib, "INHDX0")

write_cell_testbench(cell=cell, v_sup=1.8, temperature=175, tran_time=12e-9, path=path)

### Testbenches for Sequential Cells

In [6]:
cell = select_cell(lib, "DFRRHDX1")

pins = get_ff_pins(cell)

vs = create_ff_sources(  data_in="D0"
                       , pins=pins
                       , v_high=1.8
                       , v_low=0.0
                       , data_rise=1e-12
                       , data_fall=1e-12
                       , data_delay=10e-9
                       , data_period=2
                       , data_width=1
                       , clk_rise= 1e-12
                       , clk_fall=1e-12
                       , clk_delay=2e-9
                       , clk_width=50e-9
                       , clk_period=100e-9
                       )

In [7]:
netlist = generate_ff_netlist(pins + ["0", "vdd!"], "DFRRHDX1", vs, 1e-15, 125, 500e-9)

In [8]:
with open("./netlists/tb_DFRRHDX1.scs", "w") as file:
    file.write(netlist)